
## 谷氨酸棒杆菌例子
### 1. 导入必要的包

In [1]:
import pandas as pd
from retrying import retry
import urllib
from requests.adapters import HTTPAdapter, Retry
import re,requests,gzip
import numpy as np
from tqdm import tqdm
from io import StringIO
import config as cfg
from tkinter import _flatten
import tools.funclib as funclib
from tools import embdding_onehot as onehotebd
from tools import filetool as ftool
import benchmark_evaluation as eva

### 2. 定义操作函数

In [2]:

re_next_link = re.compile(r'<(.+)>; rel="next"')
retries = Retry(total=5, backoff_factor=0.25, status_forcelist=[500, 502, 503, 504])
session = requests.Session()
session.mount("https://", HTTPAdapter(max_retries=retries))

def get_next_link(headers):
    if "Link" in headers:
        match = re_next_link.match(headers["Link"])
        if match:
            return match.group(1)

def get_batch(batch_url):
    while batch_url:
        response = session.get(batch_url)
        response.raise_for_status()
        total = response.headers["x-total-results"]
        yield response, total
        batch_url = get_next_link(response.headers)


@retry(stop_max_attempt_number=10, wait_random_min=10, wait_random_max=20)   
def grab_sample_data():
    """ 通过API， 使用Uniprot ID 获取序列信息
    """

    url = f'https://rest.uniprot.org/uniprotkb/search?query=organism_id:196627&format=tsv&fields='
    url = url + f'accession,id,protein_name,sequence,length,ec,reviewed,gene_names,organism_name,annotation_score,lit_pubmed_id,date_created,date_modified,date_sequence_modified,version,&size=20&compressed=false'

    # print(f'开始获取网页数据, EC:{ec}')
    df_uniprot = pd.DataFrame()
    for batch, total in get_batch(url):
        df_uniprot_sb = pd.read_csv(StringIO(batch.text), sep='\t')
        df_uniprot = pd.concat([df_uniprot,df_uniprot_sb], axis=0)
    return df_uniprot.reset_index(drop=True)

def isecpredictable(ec_str, ecset):
    ec_array =[item.strip() for item in ec_str.split(',')]
    intersection = set(ec_array) & ecset

    if len(intersection) == len(ec_array):
        return True
    else:
        return False

### 3. 获取谷氨酸棒杆菌数据

In [4]:
FIRST_TIME = False

if FIRST_TIME:
    df_corynebacterium =grab_sample_data()
    df_corynebacterium = df_corynebacterium.rename(columns={ 'Entry':'id', 
                                    'Entry Name':'entry_name', 
                                    'Protein names':'protein_name', 
                                    'Sequence':'seq',
                                    'Length':'len',
                                    'EC number':'ec'
                                 })
    df_corynebacterium.to_csv(cfg.DATADIR +'df_corynebacterium.tsv', sep='\t', index=None)
else:
    df_corynebacterium = pd.read_csv(cfg.DATADIR +'corynebacterium.tsv', sep='\t')
    funclib.table2fasta(table=df_corynebacterium[['id','seq']], file_out=cfg.DATADIR+'corynebacterium.fasta')
    

Write finished


### 4. 跑基准预测
#### 4.1 Deepec

In [ ]:
! conda env create -f ../baselines/deepec/environment.yml
! conda activate deepec
! python /home/shizhenkun/codebase/DMLF/baselines/deepec/deepec.py -i /home/shizhenkun/codebase/DMLF/data/corynebacterium.fasta -o /home/shizhenkun/codebase/DMLF/results/case_gubang/deepec/

#### 4.2 Catfam

In [ ]:
! sudo apt-get install ncbi-blast+-legacy -y
! /home/shizhenkun/codebase/DMLF/baselines/catfam/source/catsearch.pl -d /home/shizhenkun/codebase/DMLF/baselines/catfam/CatFamDB/CatFam_v2.0/CatFam4D99R -i /home/shizhenkun/codebase/DMLF/data/corynebacterium.fasta -o /home/shizhenkun/codebase/DMLF/results/case_gubang/catfam/catfam.txt

#### 4.3 PRIAM

In [ ]:
! java -Xmx120G -jar /home/shizhenkun/codebase/DMLF/baselines/priam/PRIAM_search.jar -p /home/shizhenkun/codebase/DMLF/baselines/priam/PRIAM_JAN18 -i /home/shizhenkun/codebase/DMLF/data/corynebacterium.fasta -o /home/shizhenkun/codebase/DMLF/results/case_gubang/priam/ --bp /home/shizhenkun/downloads/ncbi-blast-2.11.0+/bin --np 78

#### 4.4 ECPred

In [ ]:
!java -jar /home/shizhenkun/codebase/DMLF/baselines/ECPred/ECPred.jar weighted /home/shizhenkun/codebase/DMLF/data/corynebacterium.fasta /home/shizhenkun/codebase/DMLF/baselines/ECPred/ /home/shizhenkun/codebase/DMLF/results/case_gubang/ecpred

#### 4.5 增补embedding
##### 4.5.1 计算需要embedding的数据

In [62]:
full_snap_data = df_corynebacterium[['id', 'seq']]
# loading exsisting features
if ftool.isfileExists(cfg.FILE_FEATURE_ESM0):
    feature_esm0 = pd.read_feather(cfg.FILE_FEATURE_ESM0)
    feature_esm32 = pd.read_feather(cfg.FILE_FEATURE_ESM32)
    feature_esm33 = pd.read_feather(cfg.FILE_FEATURE_ESM33)
    feature_unirep = pd.read_feather(cfg.FILE_FEATURE_UNIREP)
    feature_onehot = pd.read_feather(cfg.FILE_FEATURE_ONEHOT)
    #caculate embedding list
    needesm = full_snap_data[~full_snap_data.id.isin(list(set(feature_esm33.id)))]
    needunirep = full_snap_data[~full_snap_data.id.isin(list(set(feature_unirep.id)))]
    needonehot = full_snap_data[~full_snap_data.id.isin(list(set(feature_onehot.id)))]
else:
    needesm = full_snap_data
    needunirep = full_snap_data
    needonehot = full_snap_data

##### 4.5.2 esm

In [ ]:
from tools import embedding_esm as esmebd
if len(needesm)>0:
    tr_rep0, tr_rep32, tr_rep33 = esmebd.get_rep_multi_sequence(sequences=needesm, model='esm1b_t33_650M_UR50S',seqthres=1022)

    #merge existing
    feature_esm0 = pd.concat([feature_esm0, tr_rep0], axis=0).reset_index(drop=True)
    feature_esm32 = pd.concat([feature_esm32, tr_rep32], axis=0).reset_index(drop=True)
    feature_esm33 = pd.concat([feature_esm33, tr_rep33], axis=0).reset_index(drop=True)


    #save
    feature_esm0.to_feather(cfg.FILE_FEATURE_ESM0)
    feature_esm32.to_feather(cfg.FILE_FEATURE_ESM32)
    feature_esm33.to_feather(cfg.FILE_FEATURE_ESM33)

##### 4.5.3 Unirep

In [ ]:
if len(needunirep) > 0:
    from tools import embedding_unirep as unirep
    tr_unirep = unirep.getunirep(needunirep, 40)

    feature_unirep = pd.concat([feature_unirep, tr_unirep],axis=0).reset_index(drop=True)
    feature_unirep.to_feather(cfg.FILE_FEATURE_UNIREP)

##### 4.5.4 One-hot

In [61]:
from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize(progress_bar=False)
feature_onehot = pd.read_feather(cfg.FILE_FEATURE_ONEHOT)
needonehot = full_snap_data
if len(needonehot) > 0:
    tr_unirep = onehotebd.get_onehot(sequences=needonehot, padding=True, padding_window=1500)
    feature_onehot = pd.concat([feature_onehot, tr_unirep],axis=0).reset_index(drop=True)
    feature_onehot.to_feather(cfg.FILE_FEATURE_ONEHOT)

INFO: Pandarallel will run on 104 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### 5. 读取基准结果
#### 5.1 Blast

In [5]:
sprot = pd.read_feather(cfg.DATADIR+'uniprot/snap201802.feather')
df_corynebacterium.ec = df_corynebacterium.ec.fillna('-')
ec_sprot = _flatten([item.split(',') for item in list(set(sprot.ec_number))])
df_corynebacterium['ec_predictable']=df_corynebacterium.ec.apply(lambda x :isecpredictable(x, set(ec_sprot)))

res_diamond = funclib.getblast(train=sprot[['id','seq']], test=df_corynebacterium[['id','seq']])
res_diamond = res_diamond [['id','sseqid','pident']].merge(sprot[['id','ec_number']], how='left', left_on='sseqid', right_on='id')
res_diamond = res_diamond[['id_x', 'ec_number','pident']].rename(columns={'id_x':'id','ec_number':'ec_diamond'})
f_merge_res = df_corynebacterium.merge(res_diamond, on='id', how='left').drop_duplicates().reset_index(drop=True)
f_merge_res.ec_diamond = f_merge_res.ec_diamond.fillna('-')

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd --quiet
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet


#### 5.2 ECPred

In [6]:
res_ecpred = pd.read_csv(f'{cfg.RESULTSDIR}case_gubang/ecpred/ecpred.tsv', sep='\t', header=0)
res_ecpred=res_ecpred.rename(columns={'Protein ID':'id','EC Number':'ec_ecpred'})[['id','ec_ecpred']]
res_ecpred.ec_ecpred = np.where(res_ecpred.ec_ecpred=='no Prediction', '-', res_ecpred.ec_ecpred)
res_ecpred.ec_ecpred = np.where(res_ecpred.ec_ecpred=='non Enzyme', '-', res_ecpred.ec_ecpred)

f_merge_res = f_merge_res.merge(res_ecpred, on='id', how='left')
f_merge_res.ec_ecpred = f_merge_res.ec_ecpred.fillna('-')

#### 5.3 DeepEC

In [7]:
res_deepec = pd.read_csv(f'{cfg.RESULTSDIR}case_gubang/deepec/DeepEC_Result.txt', sep='\t',names=['id', 'ec_number'], header=0 )
res_deepec.ec_number=res_deepec.apply(lambda x: x['ec_number'].replace('EC:',''), axis=1)
res_deepec.columns = ['id','ec_deepec']

res = []
for index, group in  res_deepec.groupby('id'):
    if len(group)==1:
        res = res + [[group.id.values[0], group.ec_deepec.values[0]]]
    else:
        ecs_str = ','.join(group.ec_deepec.values)
        res = res +[[group.id.values[0],ecs_str]] 
res_deepec = pd.DataFrame(res, columns=['id', 'ec_deepec'])

f_merge_res = f_merge_res.merge(res_deepec, how='left', on='id')
f_merge_res.ec_deepec = f_merge_res.ec_deepec.fillna('-')

#### 5.4 CatFam

In [8]:
res_catfam = pd.read_csv(cfg.RESULTSDIR+'case_gubang/catfam/catfam.txt', sep='\t', names=['id', 'ec_catfam'])
res = []
for index, group in  res_catfam.groupby('id'):
    if len(group)==1:
        res = res + [[group.id.values[0], group.ec_catfam.values[0]]]
    else:
        ecs_str = ','.join(group.ec_catfam.values)
        res = res +[[group.id.values[0],ecs_str]] 
res_catfam = pd.DataFrame(res, columns=['id', 'ec_catfam'])
res_catfam = res_catfam.fillna('-')


f_merge_res = f_merge_res.merge(res_catfam, on='id', how='left')
f_merge_res.ec_catfam = f_merge_res.ec_catfam.fillna('-')

#### 5.5 PRIAM

In [9]:
res_priam = eva.load_praim_res(resfile=cfg.RESULTSDIR+'case_gubang/priam/PRIAM_20221104104833/ANNOTATION/sequenceECs.txt')
f_merge_res = f_merge_res.merge(res_priam, on='id', how='left')
f_merge_res.ec_priam = f_merge_res.ec_priam.fillna('-')

### 10. DMLF-ours

In [10]:
res_dmlf = pd.read_csv(cfg.RESULTSDIR+'case_gubang/dmlf/corynebacterium_pred.tsv', sep='\t')

# res_dmlf['ec_ours']=res_dmlf.ec_ours.apply(lambda x: x.split(',')[0].replace('(','').replace('\'',''))
# res_dmlf['listpred_20'] = res_dmlf.listpred_20.apply(lambda x: make_rank_array(x))
f_merge_res = f_merge_res.merge(res_dmlf, on='id', how='left')
# f_merge_res['is_full_level_ec']=f_merge_res.ec_number.apply(lambda x: False if '-' in x else True)
# f_merge_res=f_merge_res.drop_duplicates().reset_index(drop=True)

In [12]:
f_merge_res

,id,entry_name,protein_name,seq,len,ec,Reviewed,Gene Names,Organism,Annotation,...,Entry version,ec_predictable,ec_diamond,pident,ec_ecpred,ec_deepec,ec_catfam,ec_priam,ec_ours,pred_top20
0,Q79VD7,COX1_CORGL,Cytochrome c oxidase subunit 1 (EC 7.1.1.9) (C...,MTAVAPRVDGHVAPQRPEPTGHARKGSKAWLMMTTTDHKQLGIMYI...,584,7.1.1.9,reviewed,ctaD Cgl2523 cg2780,Corynebacterium glutamicum (strain ATCC 13032 ...,5.0,...,130,False,1.9.3.1,100.0,1.9.3.1,1.9.3.1,1.9.3.1,"1.9.3.1, 1.10.3.10, 3.1.3.16",1.9.3.1,"1.9.3.1(0.9978), 1.10.3.-(0.0359), 1.10.3.10(0..."
1,Q8NRC3,ODO12_CORGL,2-oxoglutarate dehydrogenase E1/E2 component (...,MSSASTFGQNAWLVDEMFQQFQKDPKSVDKEWRELFEAQGGPNTTP...,1221,1.2.4.2; 2.3.1.61,reviewed,odhA Cgl1129 cg1280,Corynebacterium glutamicum (strain ATCC 13032 ...,5.0,...,129,False,"2.3.1.61,1.2.4.2",100.0,1.2.4.2,1.2.4.2,-,"2.3.1.61, 1.2.4.2, 1.2.4.2, 2.2.1.5, 2.3.1.61,...","1.2.4.2,2.3.1.61,2.2.1.5,4.1.1.71","1.2.4.2(0.9986), 2.3.1.61(0.9962), 2.2.1.5(0.9..."
2,Q8NN33,MDH_CORGL,Malate dehydrogenase (EC 1.1.1.37),MNSPQNVSTKKVTVTGAAGQISYSLLWRIANGEVFGTDTPVELKLL...,328,1.1.1.37,reviewed,mdh Cgl2380 cg2613,Corynebacterium glutamicum (strain ATCC 13032 ...,5.0,...,125,True,1.1.1.37,100.0,1.1.1.37,1.1.1.37,1.1.1.37,"1.1.1.37, 1.1.1.37, 1.1.1.96, 1.1.1.82",1.1.1.37,"1.1.1.37(0.9992), 1.1.1.82(0.0014), 1.1.1.299(..."
3,Q8NNJ2,ODP2_CORGL,Dihydrolipoyllysine-residue acetyltransferase ...,MAFSVEMPELGESVTEGTITQWLKSVGDTVEVDEPLLEVSTDKVDT...,675,2.3.1.12,reviewed,aceF sucB Cgl2207 cg2421,Corynebacterium glutamicum (strain ATCC 13032 ...,5.0,...,131,True,2.3.1.12,100.0,2.3.1.12,2.3.1.12,2.3.1.61,"2.3.1.12, 2.3.1.12, 2.3.1.61, 2.3.1.168, 2.3.1...",2.3.1.12,"2.3.1.12(0.9444), 2.3.1.61(0.0781), 6.4.1.3(0...."
4,P04964,DAPDH_CORGL,Meso-diaminopimelate D-dehydrogenase (DAPDH) (...,MTNIRVAIVGYGNLGRSVEKLIAKQPDMDLVGIFSRRATLDTKTPV...,320,1.4.1.16,reviewed,ddh Cgl2617 cg2900,Corynebacterium glutamicum (strain ATCC 13032 ...,5.0,...,153,True,1.4.1.16,100.0,1.-.-.-,1.4.1.16,-,"1.4.1.16, 1.2.1.59",1.4.1.16,"1.4.1.16(0.9176), 1.4.1.21(0.0804), 1.2.1.10(0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3093,Q8NUD0,Q8NUD0_CORGL,50S ribosomal protein L10,MTSLRDLLVSTNADAAIADMSAFIEETVGKQSGISGIALKGAMGAA...,147,-,unreviewed,Cgl0009,Corynebacterium glutamicum (strain ATCC 13032 ...,1.0,...,90,True,-,NaN,3.-.-.-,-,-,-,-,"3.-.-.-(0.0167), 2.3.2.-(0.0074), 2.3.1.-(0.00..."
3094,Q8NUD1,Q8NUD1_CORGL,Hypothetical membrane protein,MSNPNKKPLDPAQRFSLPTRWAMFFVGIFIMSFAIGITVHAGLGTT...,223,-,unreviewed,Cgl0008,Corynebacterium glutamicum (strain ATCC 13032 ...,1.0,...,90,True,-,NaN,1.9.3.-,-,-,-,-,"2.3.1.-(0.0111), 2.3.2.3(0.0043), 2.3.1.15(0.0..."
3095,Q8NUD2,Q8NUD2_CORGL,Hydrolase_4 domain-containing protein,MIQTYILKIQSLKWAQREMLKTTLITDTLGHVTSTEPTTTAHVPLP...,322,-,unreviewed,Cgl0007,Corynebacterium glutamicum (strain ATCC 13032 ...,1.0,...,108,True,-,NaN,1.-.-.-,-,-,-,3.1.1.-,"3.1.1.-(0.2296), 2.3.1.-(0.1521), 3.1.1.3(0.03..."
3096,Q8NUD7,Q8NUD7_CORGL,Tubulin_C domain-containing protein,MIVDNFVYLWIDFKVRILSPVCADVSPAYPQFLGSCAQGGKFRGIT...,108,-,unreviewed,Cgl0002,Corynebacterium glutamicum (strain ATCC 13032 ...,1.0,...,79,True,-,NaN,-,-,-,-,-,"3.1.4.35(0.0723), 3.4.21.-(0.0305), 3.2.1.35(0..."


In [14]:
f_merge_res.to_excel(cfg.RESULTSDIR+'case_gubang/dmlf/corynebacterium_pred_results.xlsx')

In [40]:
res_dmlf.ec_ours[0].split(',')

["[['4.2.1.19'",
 ' 0.9918]',
 " ['3.4.21.49'",
 ' 0.0359]',
 " ['3.4.24.55'",
 ' 0.0066]',
 " ['1.14.13.121'",
 ' 0.0058]',
 " ['3.5.1.60'",
 ' 0.0018]',
 " ['5.3.3.12'",
 ' 0.0014]',
 " ['2.4.1.146'",
 ' 0.0004]',
 " ['2.1.1.206'",
 ' 0.0004]',
 " ['1.4.1.17'",
 ' 0.0003]',
 " ['3.1.26.12'",
 ' 0.0002]',
 " ['1.1.1.368'",
 ' 0.0002]',
 " ['2.3.1.271'",
 ' 0.0002]',
 " ['2.7.1.180'",
 ' 1e-04]',
 " ['1.14.99.68'",
 ' 1e-04]',
 " ['6.3.2.8'",
 ' 1e-04]',
 " ['1.14.14.154'",
 ' 1e-04]',
 " ['3.2.1.135'",
 ' 1e-04]',
 " ['3.4.22.64'",
 ' 1e-04]',
 " ['2.7.1.64'",
 ' 1e-04]',
 " ['1.21.3.1'",
 ' 1e-04]]']

In [23]:
f_merge_res.head(2)

,id,entry_name,protein_name,seq,len,ec,Reviewed,Gene Names,Organism,Annotation,...,Entry version,ec_predictable,ec_diamond,pident,ec_ecpred,ec_deepec,ec_catfam,ec_priam,ec_ours,pred_top20
0,Q79VD7,COX1_CORGL,Cytochrome c oxidase subunit 1 (EC 7.1.1.9) (C...,MTAVAPRVDGHVAPQRPEPTGHARKGSKAWLMMTTTDHKQLGIMYI...,584,7.1.1.9,reviewed,ctaD Cgl2523 cg2780,Corynebacterium glutamicum (strain ATCC 13032 ...,5.0,...,130,False,1.9.3.1,100.0,1.9.3.1,1.9.3.1,1.9.3.1,"1.9.3.1, 1.10.3.10, 3.1.3.16",1.9.3.1,"1.9.3.1(0.9978), 1.10.3.-(0.0359), 1.10.3.10(0..."
1,Q8NRC3,ODO12_CORGL,2-oxoglutarate dehydrogenase E1/E2 component (...,MSSASTFGQNAWLVDEMFQQFQKDPKSVDKEWRELFEAQGGPNTTP...,1221,1.2.4.2; 2.3.1.61,reviewed,odhA Cgl1129 cg1280,Corynebacterium glutamicum (strain ATCC 13032 ...,5.0,...,129,False,"2.3.1.61,1.2.4.2",100.0,1.2.4.2,1.2.4.2,-,"2.3.1.61, 1.2.4.2, 1.2.4.2, 2.2.1.5, 2.3.1.61,...","1.2.4.2,2.3.1.61,2.2.1.5,4.1.1.71","1.2.4.2(0.9986), 2.3.1.61(0.9962), 2.2.1.5(0.9..."


In [22]:
df_corynebacterium.head(2)

,id,entry_name,protein_name,seq,len,ec,Reviewed,Gene Names,Organism,Annotation,PubMed ID,Date of creation,Date of last modification,Date of last sequence modification,Entry version,ec_predictable
0,Q79VD7,COX1_CORGL,Cytochrome c oxidase subunit 1 (EC 7.1.1.9) (C...,MTAVAPRVDGHVAPQRPEPTGHARKGSKAWLMMTTTDHKQLGIMYI...,584,7.1.1.9,reviewed,ctaD Cgl2523 cg2780,Corynebacterium glutamicum (strain ATCC 13032 ...,5.0,11382224; 11577165; 12743753; 12948626; 12446663,2005-04-26,2022-10-12,2004-07-05,130,False
1,Q8NRC3,ODO12_CORGL,2-oxoglutarate dehydrogenase E1/E2 component (...,MSSASTFGQNAWLVDEMFQQFQKDPKSVDKEWRELFEAQGGPNTTP...,1221,1.2.4.2; 2.3.1.61,reviewed,odhA Cgl1129 cg1280,Corynebacterium glutamicum (strain ATCC 13032 ...,5.0,9004499; 12743753; 12948626; 20675489; 1652263...,2007-01-23,2022-10-12,2012-11-28,129,False
